In [35]:
#CONVERING SET problem is an exercise in which we have a set of discrete segments and we want to gather them in a set in order to cover another set.
    #we want to find the optimal solution which can be selected according to the weight or the number of segments

# We don't know what is the aspect of the goal state but we have a function we can use each time to verify if the current state is the goal one

#The concept is we have two set: {sets we took} {sets we didn't take}

#Each set is represened using a numerical id (from 0 to NUM_SETS-1)

#Each state is represented as state = {sets we took} {sets we didn't take}

#with the goal_check(state) function we verify if the current solution is the goal one

#we apply the Generic Search using a LifoQueue to solve our problem (we can't use a QueueLifo and so a Dijkstra approach because we don't impose any kind of cost to the sets)


In [2]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import numpy as np

In [25]:
PROBLEM_SIZE = 8
NUM_SETS=8
#this is just a way to name our tuples
State = namedtuple('State', ['taken', 'not_taken'])

In [26]:
#this operation allows us to generate randomly the sets we will use in the problem 
SETS = tuple(np.array([random() <.5 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
#goal_check(SETS)

In [27]:
def goal_check(state):
     #The function checks whether all elements of a set are covered by the selected subsets. 
     #Uses the reduce function with the np.logical_or operator to combine all the selected subsets into a single Boolean array. 
     #Then, use the np.all function to check whether all elements of the set are covered. 
     #If all elements are covered, the function returns True, otherwise it returns False.
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [28]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [7]:
#state of example - so it represents the sets (using their numerical id) taken in the current solution 
state = (set(range(NUM_SETS)),set())

print(state)
print(SETS)

#this operation sums all the overlap along the taken sets according to the boolean value in SETS[i]  
sum([SETS[i] for i in state[0]])

({0, 1, 2, 3, 4, 5, 6, 7}, set())
(array([ True, False, False,  True, False, False, False,  True]), array([ True,  True,  True,  True, False, False,  True, False]), array([ True, False, False,  True,  True,  True,  True,  True]), array([ True,  True, False,  True,  True,  True, False,  True]), array([False, False,  True, False,  True, False,  True, False]), array([ True, False,  True, False,  True, False,  True, False]), array([ True, False, False, False, False, False,  True,  True]), array([False,  True, False, False,  True, False, False, False]))


array([6, 3, 3, 4, 5, 2, 5, 4])

In [37]:
#here the application of the Generic Search viewed in class (third pack of slides pag 18)

#This is a transcription of the professor wrote in class during the 10/10/2023 lecture. Look at the code below to see my work


frontier = LifoQueue() # we use a LifoQueue (because we don't have a concept about the costs here) instead of a PriorityQueue which would be a particular case of the Generic Search (Dijkstra's alg.)
frontier.put(State(set(), set(range(NUM_SETS)))) #the first state is the one with no taken sets

counter = 0 #counter used just to count the number of occurrencies needed to solve the problem
current_state = frontier.get()  #start the resolution taking the first element from the frontier queue
while not goal_check(current_state):    #iterate until the problem is not resolved
    counter += 1
    for action in current_state[1]: #an ACTION is represented as the activity of taking one set from 
        # The ^ operator in Python is a bitwise XOR (exclusive OR) operator. It returns True if and only if its arguments differ (one is True, the other is False)
        #so here it equals to take an action (set) from not_taken and put it into taken
        # new_state = State(current_state.taken | {action}, current_state.not_taken - {action}) -> this would be the same
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})

        #it puts all the states generated into the frontier queue
        frontier.put(new_state)
    
    #endly it takes one state at time and analyze its condition (if can be considered a goal state in the while above there)
    current_state = frontier.get()

print(f"Solved in {counter:,} steps")
current_state

Solved in 3 steps


State(taken={5, 6, 7}, not_taken={0, 1, 2, 3, 4})

In [31]:
goal_check(current_state)

True

In [ ]:
#You can find my code below this line

In [29]:
# but now we would like to define an actual cost in order to find an OPTIMAL solution different from the one found above - this became a Dijkstra approach
# we are going to use the total number of True elements of the set we are taking
# [True False False] better than [True True False] I suppose

def path_cost(state):
    somma = 0
    for s in state[0]:
        for a in SETS[s]:
            if a == True:
                somma += 1
    return somma


In [40]:
#use as priority function the iinverse of path_cost: smaller is better
def priority_function(state):
    return -path_cost(state)

#Dijsktra logic applied though a priority function on a list (look at the frontier variable) NOT with a PriorityQueue. Some difference could appear in the

frontier = [] #I used a list instead of a PriorityQueue and then I sorted it after each iteration to apply the priority logic
frontier.append(State(frozenset(), frozenset(range(NUM_SETS)))) #the first state is the one with no taken sets -> we use the frozenset type since is compatible with the priority function sorting 
state_cost = {} # Use a dictionary to store the cost of each state
counter = 0 #counter used just to count the number of occurrencies needed to solve the problem
current_state = frontier.pop()  #start the resolution taking the first element from the frontier queue
while not goal_check(current_state):    #iterate until the problem is not resolved
    counter += 1
    for action in current_state[1]: #an ACTION is represented as the activity of taking one set from the not_taken group
        new_state = State(current_state.taken ^ frozenset({action}), current_state.not_taken ^ frozenset({action}))
        new_node = new_state # Use new_state as new_node
        if new_state not in state_cost and new_state not in frontier: # Use .queue to get a list of elements in the PriorityQueue
            # Add node to frontier and update solution
            state_cost[new_node] = path_cost(new_state) # Update state_cost dictionary
            frontier.append(new_state)
        elif new_state in frontier and state_cost[new_node] > path_cost(new_state): 
            #update node and solution
            state_cost[new_node] = path_cost(new_state) # Update state_cost dictionary
    frontier.sort(key=priority_function) #let's apply the priority sorting to make the list equal to a real priority queue
    current_state = frontier.pop()  #this pop takes off the element with the highest priority

print(f"Solved in {counter:,} steps")

print(current_state)

path_cost(current_state)


Solved in 52 steps
State(taken=frozenset({2, 3, 7}), not_taken=frozenset({0, 1, 4, 5, 6}))


10

In [44]:
#This should be the official Dijkstra solution. Look at how the number of needed iterations is lower while the result is the same

def priority_function(state):
    return path_cost(state)

import heapq

initial_state = State(frozenset(), frozenset(range(NUM_SETS)))
frontier = [] # Now we use a priority queue implemented with a heap
heapq.heappush(frontier, (priority_function(initial_state), initial_state)) # Push the first state into the frontier
state_cost = {initial_state: 0} # Initialize state_cost with the initial state
counter = 0 # Counter used just to count the number of occurrences needed to solve the problem
current_state = heapq.heappop(frontier)[1]  # Start the resolution taking the first element from the frontier queue
while not goal_check(current_state):    # Iterate until the problem is not resolved
    counter += 1
    for action in current_state[1]: # An ACTION is represented as the activity of taking one set from the not_taken group
        new_state = State(current_state.taken ^ frozenset({action}), current_state.not_taken ^ frozenset({action}))
        new_node = new_state # Use new_state as new_node
        new_cost = path_cost(new_state) # Calculate the cost of the new state using your path_cost function
        if new_state not in state_cost or new_cost < state_cost[new_state]: 
            state_cost[new_node] = new_cost # Update state_cost dictionary
            heapq.heappush(frontier, (priority_function(new_state), new_state))
    current_state = heapq.heappop(frontier)[1]  # This pop takes off the element with the highest priority

print(f"Solved in {counter:,} steps")

print(current_state)

path_cost(current_state)


Solved in 44 steps
State(taken=frozenset({2, 3, 7}), not_taken=frozenset({0, 1, 4, 5, 6}))


10